***
*Project:* Helmholtz Machine on Niche Construction

*Author:* Jingwei Liu, Computer Music Ph.D., UC San Diego
***

# <span style="background-color:darkorange; color:white; padding:2px 6px">Document 2</span> 

# Helmholtz Machine Analysis

This notebook goes through a detailed mathematical analysis of the principle of Helmholtz Machine.

*Updated:* March 23, 2023


In [1]:
import numpy as np

- Input layer $d_i$ with 10 neurons $i = 1, \dots, 10, 2^{10} = 1024$ possibilities
- Hidden layer $x_l$ wiht 8 neurons $l = 1, \dots, 8, 2^{8} = 256$ possibilities
- Cause layer $y_j$ wiht 5 neurons $j = 1, \dots, 5, 2^{5} = 32$ possibilities
- Hyper layer $z_k$ wiht 3 neurons $k = 1, \dots, 3, 2^{3} = 8$ possibilities
- Generative bias is always $1$ (**Ursatz** in Schenkerian analysis)

<img src="niche.jpg" >
<caption><center> **Figure 1**: Helmholtz Machine in Niche Construction  </center></caption>

*Note: **The mathematical formulation is derived by the author (Jingwei Liu) based on her understanding of the original paper** [The Helmholtz Machine (Dayan 1995)](https://direct.mit.edu/neco/article/7/5/889/5898/The-Helmholtz-Machine). The mathematical notations loosely correspond to that in the paper but not aligned exactly.* 

### The Recognition Distribution

The recognition model is given by the bottom-up encoding process from evidence to hidden cause. The observation/data layer $d$ (indexed by $i$) is drawn from the dataset determined by the external world, which also represents the observable variables. In Figure 1 we have 3 layers of hidden causes, where the probabilities of neuron values equal to $1$ are denoted by $q$ and the complete assignment of neuron values is denoted by $\alpha$.

***Remark***: Here we use binary neurons. One modification we did is to replace the $\{0,1\}$ binary values with $\{-1,1\}$. This is because when we update the parameters $\theta$ and $\phi$ using backpropogation, the derivatives are multiplied by the neuron values. If the neuron is assigned $0$, then the gradient descent fails to work (there is no parameter updating). This will be shown in the later part of our mathematical demonstration, and to avoid this problem, we use $-1$ to replace $0$.

The recognition probability
$$
Q(\alpha | d) = \prod_l [q_l^2]^{\frac{1+x_l}{2}} [1-q_l^2]^{\frac{1-x_l}{2}} \centerdot \prod_j [q_j^3]^{\frac{1+y_j}{2}} [1-q_j^3]^{\frac{1-y_j}{2}} \centerdot \prod_k [q_k^4]^{\frac{1+z_k}{2}} [1-q_k^4]^{\frac{1-z_k}{2}} 
$$


We choose to write it out as separate layers using distinct indices to show the calculations in a more straight forward way. Here each $q$ is a function of its previous layer and the parameters that connect them, namely

$$q_l^2(d,\phi_l^{1,2},b_{l}^{1,2}) = \sigma ( \sum_i \phi_{i,l}^{1,2}d_i + b_{l}^{1,2}) $$
$$q_j^3(x,\phi_j^{2,3},b_{j}^{2,3}) = \sigma(\sum_l \phi_{l,j}^{2,3}x_l + b_{j}^{2,3}) $$
$$q_k^4(y,\phi_k^{3,4},b_{k}^{3,4}) = \sigma(\sum_j \phi_{j,k}^{3,4}y_j + b_{k}^{3,4}) $$

where $\sigma$ is the sigmoid function $\sigma(x) = \frac{1}{1+e^{-x}}$, and $b$'s are the biases we add to each neuron (note that there is no bias term in the original paper, and the bias parameters are not shown in the graph).

*Remark*: In $Q(\alpha | d)$, it's easy to tell that when a neuron $x$ takes value $1$, the exponent $\frac{1+x_l}{2} = 1, \frac{1-x_l}{2} = 0$, thus we use the probability $q$ in the distribution; on the other hand, if $x=-1$, then $\frac{1+x_l}{2} = 0, \frac{1-x_l}{2} = 1$, then we use the probability $1-q$ instead. Every single neuron gives a Bernoulli distribution.

### The Generative Model

The generative model generates from the generative bias, which is always $1$. We hope the model (or the brain in analogy) can generate a repertoire that maps to the distribution of real observations. 

The generative distribution
$$
P(\alpha,d) = \prod_k [p_k^4]^{\frac{1+z_k}{2}} [1-p_k^4]^{\frac{1-z_k}{2}} \centerdot \prod_j [p_j^3]^{\frac{1+y_j}{2}} [1-p_j^3]^{\frac{1-y_j}{2}} \centerdot \prod_l [p_l^2]^{\frac{1+x_l}{2}} [1-p_l^2]^{\frac{1-x_l}{2}} \centerdot \prod_i [p_i^1]^{\frac{1+d_i}{2}} [1-p_i^1]^{\frac{1-d_i}{2}}
$$

Similarly, we can write out the formulas for $p$,

\begin{array}{rcl}
p_k^4(\theta_k) &= &\sigma(\theta_k) \\
p_j^3(z,\theta_j^{4,3},b_{j}^{4,3}) &= &\sigma(\sum_k \theta_{k,j}^{4,3}z_k + b_{j}^{4,3}) \\
p_l^2(y,\theta_l^{3,2},b_{l}^{3,2}) &= &\sigma(\sum_j \theta_{j,l}^{3,2}y_j + b_{l}^{3,2}) \\
p_i^1(x,\theta_i^{2,1},b_{i}^{2,1}) &= &\sigma(\sum_l \theta_{l,i}^{2,1}x_l + b_{i}^{2,1}) \\
\end{array}

Here we abused the notation for bias a little bit. Notice that the $b$'s in the recognition model are different from that in the generation model.

### Free Energy and Variational Lower Bound

As stated in [Variational Inference](https://arxiv.org/abs/1601.00670), our goal is to compute the true posterior $P(\alpha|d)$, which is given by 

$$
P(\alpha|d) = \frac{P(\alpha,d)}{P(d)} = \frac{P(\alpha,d)}{\sum_\alpha P(\alpha,d) d\alpha}
$$

The integral over underlying causes is usually intractable (either unavailable in closed form or requires exponential time to compute), so the true posterior $P(\alpha|d)$ cannot be computed directly.

In variational inference, we use an approximate posterior $Q_\phi(\alpha|d)$ to approximate the true posterior $P(\alpha|d)$, where $Q_\phi(\alpha|d)$ belongs to a parameterized family $\mathscr{Q}_\phi$ of probability densities. Our goal is to find the member of this family that minimizes Kullback-Leibler (KL) divergence to the exact posterior,

$$
Q_\phi^*(\alpha|d) = \operatorname*{arg \, min}_{Q_\phi(\alpha|d) \in \mathscr{Q_\phi}} D_{KL}[Q_\phi(\alpha|d) || P(\alpha|d)]
$$

Note that the approxiamate densities are conditioned on $d$, where $d$ represents a single data point instead of the whole dataset. It means that we derive different hidden cause distributions for each data point, instead of a holistic distribution for the entire dataset. This also entails the different formulations of [Statistical Variational Inference](https://arxiv.org/abs/1601.00670) and [Auto-Encoding Variational Bayes](https://arxiv.org/abs/1312.6114).

The variational method kicks in with the reformulation of the KL-divergence term,

$$
\begin{align}
D_{KL}[Q_\phi(\alpha|d) || P(\alpha|d)] &= \mathbb{E}_Q[\log Q_\phi(\alpha|d)] - \mathbb{E}_Q[\log P(\alpha|d)] \\
                                        &= \mathbb{E}_Q[\log Q_\phi(\alpha|d)] - \mathbb{E}_Q[\log P_\theta(\alpha,d)] + \log P(d)
\end{align}
$$

We know the KL-divergence term is non-negative, therefore

$$
\log P(d) \ge \mathbb{E}_Q[\log P_\theta(\alpha,d)] - \mathbb{E}_Q[\log Q_\phi(\alpha|d)]
$$

The term on the right hand side, $\mathbb{E}_Q[\log P_\theta(\alpha,d)] - \mathbb{E}_Q[\log Q_\phi(\alpha|d)]$ is called the Evidence Lower Bound (ELBO). By maximizing this term we implicitly maximizes the log-evidence $\log P(d)$.

The free energy is given by the negative ELBO,

$$
F = \mathbb{E}_Q[\log Q_\phi(\alpha|d)] - \mathbb{E}_Q[\log P_\theta(\alpha,d)] = D_{KL}[Q_\phi(\alpha|d) || P_\theta(\alpha,d)]
$$

This term works as the loss function which we try to minimize in the Helmholtz machine. 

Now we plug in the recognition distribution $Q_\phi(\alpha|d)$ and generative distribution $P_\theta(\alpha,d)$ into the free energy term,

$$
\begin{align}
F &= D_{KL}[Q_\phi(\alpha|d) || P_\theta(\alpha,d)] \\
&= \mathbb{E}_Q[\log \frac{Q_\phi(\alpha|d)}{P_\theta(\alpha,d)}] \\
&= \mathbb{E}_Q[\log \frac{\prod_l [q_l^2]^{\frac{1+x_l}{2}} [1-q_l^2]^{\frac{1-x_l}{2}} \centerdot \prod_j [q_j^3]^{\frac{1+y_j}{2}} [1-q_j^3]^{\frac{1-y_j}{2}} \centerdot \prod_k [q_k^4]^{\frac{1+z_k}{2}} [1-q_k^4]^{\frac{1-z_k}{2}}}{\prod_k [p_k^4]^{\frac{1+z_k}{2}} [1-p_k^4]^{\frac{1-z_k}{2}} \centerdot \prod_j [p_j^3]^{\frac{1+y_j}{2}} [1-p_j^3]^{\frac{1-y_j}{2}} \centerdot \prod_l [p_l^2]^{\frac{1+x_l}{2}} [1-p_l^2]^{\frac{1-x_l}{2}} \centerdot \prod_i [p_i^1]^{\frac{1+d_i}{2}} [1-p_i^1]^{\frac{1-d_i}{2}}}] \\
&= \mathbb{E}_Q[\log \prod_k \left[\frac{q_k^4}{p_k^4}\right]^{\frac{1+z_k}{2}} \left[ \frac{1-q_k^4}{1-p_k^4}\right]^{\frac{1-z_k}{2}} \centerdot \prod_j \left[\frac{q_j^3}{p_j^3}\right]^{\frac{1+y_j}{2}} \left[\frac{1-q_j^3}{1-p_j^3}\right]^{\frac{1-y_j}{2}} \centerdot \prod_l \left[\frac{q_l^2}{p_l^2}\right]^{\frac{1+x_l}{2}} \left[\frac{1-q_l^2}{1-p_l^2}\right]^{\frac{1-x_l}{2}} \centerdot \prod_i \left[\frac{1}{p_i^1}\right]^{\frac{1+d_i}{2}} \left[\frac{1-0}{1-p_i^1}\right]^{\frac{1-d_i}{2}}] \\
&= \mathbb{E}_Q[\sum_k \left( \frac{1+z_k}{2} \log \left[\frac{q_k^4}{p_k^4}\right] + \frac{1-z_k}{2} \log \left[ \frac{1-q_k^4}{1-p_k^4}\right] \right) + \sum_j \left( \frac{1+y_j}{2} \log \left[\frac{q_j^3}{p_j^3}\right] + \frac{1-y_j}{2} \log \left[ \frac{1-q_j^3}{1-p_j^3}\right] \right) + \sum_l \left( \frac{1+x_l}{2} \log \left[\frac{q_l^2}{p_l^2}\right] + \frac{1-x_l}{2} \log \left[ \frac{1-q_l^2}{1-p_l^2}\right] \right) + \sum_i \left( \frac{1+d_i}{2} \log \left[\frac{1}{p_i^1}\right] + \frac{1-d_i}{2} \log \left[ \frac{1-0}{1-p_i^1}\right] \right)] \\
\end{align}
$$

From this formulation we can roughly tell that if we make every corresponding $q_n^m$ and $p_n^m$ equal (or approaching each other), the free energy will be minimized. Namely $Q_\alpha = P_\alpha$ at the optimal end point, if it can be reached.

### Wake-Sleep Algorithm

[The Wake-Sleep Algorithm](https://www.cs.toronto.edu/~hinton/absps/ws.pdf) is designed as an easy-to-compute parameter updating mechanism for Helmholtz machine. In the wake phase, we perform the bottom-up recognition process using the current weights $\phi$ to get an instance of complete neuron assignments $\alpha$ and its corresponding recognition probabilities $q$. Then we update the generative weights $\theta$ based on the wake phase using gradient descent. In the sleep phase, we do the opposite. We instead fix the generative weights and update the recognition weights based on the generative process. In this algorithm, the approxiamate posterior and true generative distribution try to approxiamate each other alternatively, driving the model to converge to its [minimum description length](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=488312). 

Again, the free energy is formulated as follows
$$
F = \mathbb{E}_Q[\log Q_\phi(\alpha|d)] - \mathbb{E}_Q[\log P_\theta(\alpha,d)] 
$$

In wake phase, the recognition distribution $Q$ is fixed and the parameters we aim to update are only involved with $P$. Therefore, the derivative $\frac{\partial F}{\partial \theta}$ is independent of the first term on the right hand side. We only need to compute the derivatives of the second term.

We write out the second term

$$
\begin{align}
-\mathbb{E}_Q[\log P_\theta(\alpha,d)] &= -\sum_\alpha Q_\phi(\alpha|d)\log P_\theta(\alpha,d) \\
&= -\sum_\alpha Q_\phi(\alpha|d) \sum_k \left( \frac{1+z_k}{2} \log \left[p_k^4\right] + \frac{1-z_k}{2} \log \left[ 1-p_k^4\right] \right) + \sum_j \left( \frac{1+y_j}{2} \log \left[p_j^3\right] + \frac{1-y_j}{2} \log \left[ 1-p_j^3\right] \right) + \sum_l \left( \frac{1+x_l}{2} \log \left[p_l^2\right] + \frac{1-x_l}{2} \log \left[1-p_l^2\right] \right) + \sum_i \left( \frac{1+d_i}{2} \log \left[p_i^1\right] + \frac{1-d_i}{2} \log \left[1-p_i^1\right] \right)
\end{align}
$$

Let $m$ denote the layer and $n$ denote the chosen neuron in layer $m$, we calculate the derivative

$$
\begin{align}
\frac{\partial F}{\partial p^m_n} &= -\frac{\partial \ \mathbb{E}_Q[\log P_\theta(\alpha,d)]}{\partial p^m_n} \\
&= -\sum_\alpha Q_\phi(\alpha|d) \left( \frac{1+s_n^m}{2} \centerdot \frac{1}{p_n^m} - \frac{1-s_n^m}{2} \centerdot \frac{1}{1-p_n^m}\right)
\end{align}
$$

We know $p_n^m = \sigma(\sum_k \theta_{k,n}^{m+1,m} \centerdot s_k^{m+1} + b_{n}^{m+1,m})$, and the derivative of sigmoid function is $\sigma'(x) = \sigma(x)(1-\sigma(x))$. Therefore

\begin{array}{rcl}
\frac{\partial p_n^m}{\partial \theta_{k,n}^{m+1,m}} &= &p_n^m(1-p_n^m)s_k^{m+1} \\
\frac{\partial p_n^m}{\partial b_{n}^{m+1,m}} &= &p_n^m(1-p_n^m)
\end{array}

By the chain rule, we have

\begin{array}{rrrrl}
\frac{\partial F}{\partial \theta_{k,n}^{m+1,m}} & = &\frac{\partial F}{\partial p^m_n} \centerdot \frac{\partial p_n^m}{\partial \theta_{k,n}^{m+1,m}} &= & -\sum_\alpha Q_\phi(\alpha|d)s_k^{m+1} \left( \frac{1+s_n^m}{2}(1-p_n^m) - \frac{1-s_n^m}{2}p_n^m \right)\\
\frac{\partial F}{\partial b_{n}^{m+1,m}} &=& \frac{\partial F}{\partial p^m_n} \centerdot \frac{\partial p_n^m}{\partial b_{n}^{m+1,m}} &= & -\sum_\alpha Q_\phi(\alpha|d) \left( \frac{1+s_n^m}{2}(1-p_n^m) - \frac{1-s_n^m}{2}p_n^m \right)
\end{array}

The updating rule of the generative parameters is gradient descent, namely

\begin{array}{rcl}
\theta_{k,n}^{m+1,m} &=& \theta_{k,n}^{m+1,m} - lr \centerdot \frac{\partial F}{\partial \theta_{k,n}^{m+1,m}} \\
b_{n}^{m+1,m} &=& b_{n}^{m+1,m} - lr \centerdot \frac{\partial F}{\partial b_{n}^{m+1,m}}
\end{array}

where $lr$ denotes the learning rate.

Theoretically, to calculate $\frac{\partial F}{\partial \theta_{k,n}^{m+1,m}}$, we are supposed to sum over all possible explanations $\alpha$ weighted by $Q_\phi(\alpha|d)$ of the local delta rule. However, this process is computationally expensive thus infeasible. Instead, we use the stochastic gradient descent to approxiamate the true gradient by the gradient at a single instance sampled from the bottom-up recognition process over one sweep:

$$
\begin{align}
\frac{\partial F}{\partial \theta_{k,n}^{m+1,m}} &= -Q_\phi(\alpha|d)s_k^{m+1} \left( \frac{1+s_n^m}{2}(1-p_n^m) - \frac{1-s_n^m}{2}p_n^m \right) \\
&= Q_\phi(\alpha|d)s_k^{m+1} \left(p_n^m - \frac{1+s_n^m}{2} \right)   \\
&= \left \{
\begin{array}{ll}
-Q_\phi(\alpha|d)s_k^{m+1}(1-p_n^m) & \text{if } s_n^m = 1 \\
Q_\phi(\alpha|d)s_k^{m+1} \centerdot p_n^m & \text{if } s_n^m = -1
\end{array} \right.
\end{align}
$$

*Remark 1* : If we take out the $Q_\phi(\alpha|d)$ term, the derivative $\frac{\partial F}{\partial \theta_{k,n}^{m+1,m}}$ takes exactly the same form as the local delta rule given in [The Wake-Sleep Algorithm ](https://www.cs.toronto.edu/~hinton/absps/ws.pdf) equation (4). It turns out that whether we use binary values $\{0,1\}$ or $\{-1,1\}$, the parameter updatig rule stays the same. It's easy to understand since the parameters $\theta$ and $\phi$ determine the **probability** of neuron values which is not dependent on the specific values taken.

*Remark 2* : Now we can clearly see why we propose to replace the classical binary values $\{0,1\}$ with $\{-1,1\}$. The parameter updating is based on the computation of its derivative $\frac{\partial F}{\partial \theta_{k,n}^{m+1,m}}$. However, according to the local delta rule, $s_k^{m+1}$ functions as a multiplier in the derivative. Therefore, when $s_k^{m+1} = 0$, the partial derivative equals to $0$, so the parameter is not updated. This will hinder the learning process as the learning only takes effect when $s_k^{m+1} = 1$. This situation could be solved by using any binary value not including $0$. For convenience, here we use $-1$ to replace $0$ so that the parameter is always updated by appropriate amount whether the neuron $s_k^{m+1}$ takes value $1$ or $-1$.

*Remark 3* : Another modification we did is to multiply the probability density $Q_\phi(\alpha|d)$ over the local delta rule. Intuitively, the sampled explanation $\alpha$ belongs to the exponential number of possible explanations over $d$ and it's not necessarily the maximum likelihood explanation. By multiplying the probability of the sampled cause $\alpha$, we control the parameter updating amount propotional to this probability. Namely, when the sampled explanation is more possible, we update the parameters (in a large step) more inclined to this explanation; on the other hand, when the sampled explanation is less possible, we barely update the parameters in a way not favoring this explanation.

Now let's get back to the original formula of free energy and take the analysis of local delta rule used for parameter updating in classical Helmholtz machine one step further. Our goal is to minimize $F$ with respect to generative parameters $\theta$ (and the bias), 

$$
\begin{align}
\operatorname*{arg \, min}_\theta F &= \operatorname*{arg \, min }_\theta \, (\mathbb{E}_Q[\log Q_\phi(\alpha|d)] - \mathbb{E}_Q[\log P_\theta(\alpha,d)]) \\
&= \operatorname*{arg \, min }_\theta \, (-\mathbb{E}_Q[\log P_\theta(\alpha,d)]) \\
&= \operatorname*{arg \, max }_\theta \, \mathbb{E}_Q[\log P_\theta(\alpha,d)] \\
&= \operatorname*{arg \, max }_\theta \, (\sum_\alpha Q_\phi(\alpha|d)\log P_\theta(\alpha,d)) \\
\end{align}
$$

Now we use a single sampled instance $\alpha$ to compute this term instead of summing over the entire latent space, then the optimization target becomes

$$
\operatorname*{arg \, max }_\theta \, [Q_\phi(\alpha|d)\log P_\theta(\alpha,d)] = \operatorname*{arg \, max }_\theta \log P_\theta(\alpha,d)
$$

If we calculate the derivative $\frac{\partial \log P_\theta(\alpha,d)}{\partial \theta}$, it corresponds to the local delta rule exactly.

Theoretically, we aim to minimize the variational free energy, which is constructed as a perfect integral optimization target for the entire system. However, in the downstream task, it's difficult to compute the derivatives directly, and that's why we introduced the wake-sleep algorithm to update the parameters alternatively. Moreover, for simplicity, we choose to use a single sampled $\alpha$ to replace the holistic distribution. These simplification and approxiamation finally altered our optimization target from $D_{KL}[Q_\phi(\alpha|d) || P_\theta(\alpha,d)]$ to $\log P_\theta(\alpha,d)$. Given fixed $\alpha$ and $d$, we maximize their joint probability, which is to say that we update $\theta$ to increase the chance of co-occurance of this specific explanation $\alpha$ and its evidence $d$. This is exactly what the local delta rule is forcing.


Now we have two learning rules, one is the **pure local delta rule**, where

$$
\frac{\partial F}{\partial \theta_{k,n}^{m+1,m}} = \left \{
\begin{array}{ll}
-s_k^{m+1}(1-p_n^m) & \text{if } s_n^m = 1 \\
s_k^{m+1} \centerdot p_n^m & \text{if } s_n^m = -1
\end{array} \right.
$$

and the other is **weighted local delta rule**, where the local delta rule is weighted by the recognition distribution $Q_\phi(\alpha|d)$,

$$
\frac{\partial F}{\partial \theta_{k,n}^{m+1,m}} = \left \{
\begin{array}{ll}
-Q_\phi(\alpha|d)s_k^{m+1}(1-p_n^m) & \text{if } s_n^m = 1 \\
Q_\phi(\alpha|d)s_k^{m+1} \centerdot p_n^m & \text{if } s_n^m = -1
\end{array} \right.
$$

We will see the two rules have different applications in the learning process. Now let's step back a little bit from the mathematical deduction of derivatives. If we look into the local delta rule, we see that to put $\frac{\partial F}{\partial \theta_{k,n}^{m+1,m}}$ near $0$, we drive the probability $p_n^m$ to $1$ when $s_n^m = 1$ and $p_n^m$ to $0$ when $s_n^m = -1$. This fits our intuition because $p_n^m$ denotes the probability of $s_n^m = 1$ originally. However, if we go back to the last statement in the section *Free Energy and Variational Lower Bound*, we see that the optimal end point of minimizing free energy would be $Q_\alpha = P_\alpha$, namely $p_n^m = q_n^m$. Therefore, the local delta rule seems too harsh. It drives $p_n^m$ to the extreme of probability ($1$ or $0$), instead of the distribution optimum $q_n^m$. The reason is discussed above. By modification and simplification of the optimization goal, we finally end up maximizing the joint probability of given $\alpha$ and $d$ where the recognition distribution $Q$ doesn't play a role. To maximize $\log P_\theta(\alpha,d)$, it's natural that we put all weights on the "correct" classifier specified by this instance thus driving the system towards extreme.

Notice that weighting the local delta rule by the recognition distribution $Q_\phi(\alpha|d)$ doesn't alter the convergence point for $p_n^m$. Intuitively, we would change the updating rule from $(\frac{1+s_n^m}{2} - p_n^m)$ to $(q_n^m - p_n^m)$. But how to derive it mathematically? In [the Helmholtz Machine](https://direct.mit.edu/neco/article/7/5/889/5898/The-Helmholtz-Machine), equation (3.11) gives an expression that averages the neuron values over $\alpha$ in the entire latent space instead of converging to a single sampled point, namely

$$
\sum_m\sum_n\left( \frac{1+s_n^m}{2} \log \left[p_n^m \right] + \frac{1-s_n^m}{2} \log \left[1-p_n^m \right] \right) \to \sum_m\sum_n\left[ q_n^m \log \left[p_n^m \right] + (1-q_n^m) \log \left[1-p_n^m \right] \right]
$$

Computing the derivative, we get another parameter updating rule we call **KL-Divergence rule**, where

$$
\frac{\partial F}{\partial \theta_{k,n}^{m+1,m}} = s_k^{m+1}(p_n^m - q_n^m)
$$

In the sleep phase, we generate an instance using the current generative weights then update the recognition weights based on it. The process is exactly the same as the procedure we discussed for the wake phase, as long as we modify the free energy to a slightly different version where the relative positions of $P$ and $Q$ are exchanged, namely

$$
\tilde{F} = \mathbb{E}_P[\log P_\theta(\alpha,d)] - \mathbb{E}_P[\log Q_\phi(\alpha|d)] 
$$

The computation of the derivative $\frac{\partial \tilde{F}}{\partial \phi}$ follows the same procedure discussed above. We will omit the duplicative process here and list the results directly.

- **Pure local delta rule**

$$
\frac{\partial \tilde{F}}{\partial \phi_{k,n}^{m-1,m}} = \left \{
\begin{array}{ll}
-s_k^{m-1}(1-q_n^m) & \text{if } s_n^m = 1 \\
s_k^{m-1} \centerdot q_n^m & \text{if } s_n^m = -1
\end{array} \right.
$$

- **Weighted local delta rule**

$$
\frac{\partial \tilde{F}}{\partial \phi_{k,n}^{m-1,m}} = \left \{
\begin{array}{ll}
-P_\theta(\alpha,d)s_k^{m-1}(1-q_n^m) & \text{if } s_n^m = 1 \\
P_\theta(\alpha,d)s_k^{m-1} \centerdot q_n^m & \text{if } s_n^m = -1
\end{array} \right.
$$

- **Expectation rule**

$$
\frac{\partial \tilde{F}}{\partial \phi_{k,n}^{m-1,m}} = s_k^{m-1}(q_n^m - p_n^m)
$$

*Remark 1:* The parameter updating rules are computed based on the assumption of mutual independency among $p$'s and $q$'s across layers. It means that the information required to modify each neuron is local. The backpropogation doesn't go through multiple layers to determine the weights for the current single neuron.

*Remark 2:* 

In [13]:
# Parameter initialization
Phi_12 = np.random.rand(8,10)
Phi_23 = np.random.rand(5,8)
Phi_34 = np.random.rand(3,5)
b_12 = np.random.rand(8)
b_23 = np.random.rand(5)
b_34 = np.random.rand(3)

Theta = np.random.rand(3,1)
Theta_43 = np.random.rand(5,3)
Theta_32 = np.random.rand(8,5)
Theta_21 = np.random.rand(10,8)
b_43 = np.random.rand(5)
b_32 = np.random.rand(8)
b_21 = np.random.rand(10)

Total number of parameters: 135 + 16 + 138 + 23 = 312

In [4]:
def sigmoid(x):
    y = 1/(1+np.exp(-x))
    return y

- Major improvement: we use binary representatioin {1,-1} instead of {0,1} to replace the local delta rule where gradients vanish when the neuron takes value 0
- We use **rejection sampling** for each layer to choose from -1 or 1 (for computation efficiency)

In [22]:
def wake_forward(data,Phi_12,Phi_23,Phi_34,b_12,b_23,b_34):
    q_2 = sigmoid(np.matmul(Phi_12,data) + b_12)
    x = ((q_2 > np.random.rand(8)).astype(int) - 0.5)*2    # rejection sampling
    
    q_3 = sigmoid(np.matmul(Phi_23,x) + b_23)
    y = ((q_2 > np.random.rand(5)).astype(int) - 0.5)*2
    
    q_4 = sigmoid(np.matmul(Phi_34,y) + b_34)
    z = ((q_2 > np.random.rand(3)).astype(int) - 0.5)*2
    
    Q_2 = np.cumprod(q_2[np.where(x == 1)])[-1] * np.cumprod(1-q_2[np.where(x == -1)])[-1]*(2**q_2.size)
    Q_3 = np.cumprod(q_3[np.where(x == 1)])[-1] * np.cumprod(1-q_3[np.where(x == -1)])[-1]*(2**q_3.size)
    Q_4 = np.cumprod(q_4[np.where(x == 1)])[-1] * np.cumprod(1-q_4[np.where(x == -1)])[-1]*(2**q_4.size)
    Q = Q_2 * Q_3 * Q_4
    
    return q_2,q_3,q_4,x,y,z,Q

In [23]:
def sleep_forward(Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21):
    p_4 = sigmoid(Theta)
    z = ((p_4 > np.random.rand(3)).astype(int) - 0.5)*2
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    y = ((p_3 > np.random.rand(5)).astype(int) - 0.5)*2
    
    p_2 = sigmoid(np.matmul(Theta_32,y) + b_32)
    x = ((p_2 > np.random.rand(8)).astype(int) - 0.5)*2
    
    p_1 = sigmoid(np.matmul(Theta_21,x) + b_21)
    d = ((p_1 > np.random.rand(10)).astype(int) - 0.5)*2
    
    P_4 = np.cumprod(p_4[np.where(x == 1)])[-1] * np.cumprod(1-p_4[np.where(x == -1)])[-1]*(2**p_4.size)
    P_3 = np.cumprod(p_3[np.where(x == 1)])[-1] * np.cumprod(1-p_3[np.where(x == -1)])[-1]*(2**p_3.size)
    P_2 = np.cumprod(p_2[np.where(x == 1)])[-1] * np.cumprod(1-p_2[np.where(x == -1)])[-1]*(2**p_2.size)
    P_1 = np.cumprod(p_1[np.where(x == 1)])[-1] * np.cumprod(1-p_1[np.where(x == -1)])[-1]*(2**p_1.size)
    P = P_1 * P_2 * P_3 * P_4
    
    return p_4,p_3,p_2,p_1,z,y,x,d,P

In [ ]:
def wake_update_delta(z,y,x,d,Q,Theta,Theta_43,Theta_32,Theta_21,b_43,b_32,b_21,lr):
    p_4 = sigmoid(Theta)
    Theta -= lr * Q * (p_4 - (1+z)/2)
    
    p_3 = sigmoid(np.matmul(Theta_43,z) + b_43)
    Theta_43 -= lr * Q * np.outer(((1+y)/2 - p_3), z)
    b_43 -= lr * Q * (p_3 - (1+y)/2)
    
    

In [7]:
x = np.array([0,1,2,3])
y = sigmoid(x)
y

array([0.5       , 0.73105858, 0.88079708, 0.95257413])

In [8]:
data = np.array([-1,1,1,1,-1,-1,1,-1,1,1])

In [28]:
q2 = np.array([.5,.6,.3,.4,.8,.2,.1,.7,.9,.3])

In [29]:
q2[np.where(data == 1)]

array([0.6, 0.3, 0.4, 0.1, 0.9, 0.3])

In [37]:
np.cumprod(q2[np.where(data == 1)])[-1]*(2**6)

0.12441599999999998

In [36]:
np.cumprod(q2)

array([5.00000e-01, 3.00000e-01, 9.00000e-02, 3.60000e-02, 2.88000e-02,
       5.76000e-03, 5.76000e-04, 4.03200e-04, 3.62880e-04, 1.08864e-04])

In [10]:
np.matmul(Phi_12,data).shape

(8,)

In [41]:
q2[np.where(data == -1)]

array([0.5, 0.8, 0.2, 0.7])

In [40]:
np.cumprod(1-q2[np.where(data == -1)])

array([0.5  , 0.1  , 0.08 , 0.024])

In [45]:
q2.size

10

In [12]:
np.random.rand(3)

array([0.89922706, 0.53727025, 0.69886637])

In [14]:
a = np.random.rand(8)
b = np.random.rand(8)
a

array([0.56841322, 0.51266799, 0.08058734, 0.40352704, 0.39380246,
       0.42456678, 0.41012379, 0.85273789])

In [15]:
b

array([0.90475718, 0.49986678, 0.71296529, 0.61900869, 0.49170206,
       0.86393567, 0.36297424, 0.68769113])

In [17]:
(a > b).astype(int)

array([0, 1, 0, 0, 0, 0, 1, 1])

In [18]:
((a > b).astype(int) - 0.5) * 2

array([-1.,  1., -1., -1., -1., -1.,  1.,  1.])

In [24]:
1-a

array([0.43158678, 0.48733201, 0.91941266, 0.59647296, 0.60619754,
       0.57543322, 0.58987621, 0.14726211])

In [46]:
(a > b).astype(int).shape

(8,)

In [47]:
np.outer(a,b)

array([[0.51427594, 0.28413089, 0.4052589 , 0.35185272, 0.27948995,
        0.49107246, 0.20631935, 0.39089273],
       [0.46384004, 0.2562657 , 0.36551448, 0.31734594, 0.25207991,
        0.44291216, 0.18608527, 0.35255723],
       [0.07291198, 0.04028293, 0.05745598, 0.04988426, 0.03962496,
        0.06962228, 0.02925113, 0.0554192 ],
       [0.36509398, 0.20170976, 0.28770077, 0.24978674, 0.19841508,
        0.3486214 , 0.14646992, 0.27750196],
       [0.3562956 , 0.19684877, 0.28076748, 0.24376714, 0.19363348,
        0.34021999, 0.14294015, 0.27081446],
       [0.38412984, 0.21222683, 0.30270138, 0.26281052, 0.20876036,
        0.36679838, 0.1541068 , 0.29197081],
       [0.37106244, 0.20500726, 0.29240403, 0.25387019, 0.20165871,
        0.35432057, 0.14886437, 0.28203849],
       [0.77152073, 0.42625534, 0.60797252, 0.52785216, 0.41929298,
        0.73671068, 0.30952188, 0.58642028]])